In [1]:
!pip install emoji

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 KB 731.5 kB/s eta 0:00:00a 0:00:01


In [40]:
import pandas as pd
import numpy as np
import emoji
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input,AveragePooling1D,Dense,Flatten,Embedding, LSTM, Bidirectional
from tensorflow.keras.models import Model

In [4]:
emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

In [5]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[str(label)],language='alias')

In [7]:
dir = '/home/milad/Desktop/python/AI/DL/Mini_project/db/emojify'
df = pd.read_csv(f'{dir}/train_emoji.csv',header=None)
df.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [8]:
X_train,Y_train = df[0].values,df[1].values

In [9]:
test_df = pd.read_csv(f'{dir}/test_emoji.csv',header=None)
X_test,Y_test = test_df[0].values,test_df[1].values

In [10]:
X_test.shape,Y_test.shape

((56,), (56,))

In [12]:
index=2

print(X_train[index],label_to_emoji(Y_train[index]))

It is the worst day in my life 😞


In [14]:
Y_train_oh=to_categorical(Y_train,5)
Y_test_oh=to_categorical(Y_test,5)

In [16]:
Y_test_oh[1]

array([0., 0., 0., 1., 0.], dtype=float32)

In [17]:
X_train[0]

'never talk to me again'

In [18]:
max(X_train,key=len),len(max(X_train,key=len))

('I am so impressed by your dedication to this project', 52)

In [19]:
max_len=len(max(X_train,key=len).split())
max_len

10

In [21]:
dir = '/home/milad/Desktop/python/AI/DL/Mini_project/db/glove.6B'
embeddings_index = {}
words_to_index = {}
index_to_words = {}
with open(f'{dir}/glove.6B.100d.txt',encoding = 'utf-8') as f:
  lines = f.readlines()
  words = set()
  for line in lines:
    values = line.split()
    word = values[0]
    embed = values[1:]
    weights = np.asarray(embed,dtype = 'float32')
    embeddings_index[word] = weights
    words.add(word)

  i = 1
  for w in sorted(words):
    words_to_index[w] = i
    index_to_words[i] = w
    i += 1

In [22]:
def senetences_to_indices(X,word_to_idx,max_len):
  m = X.shape[0]
  X_indices = np.zeros((m,max_len))
  for i in range(m):
    sentence_words = X[i].lower().split()
    for j,w in enumerate(sentence_words):
      X_indices[i,j] = word_to_idx[w]
  return X_indices

In [24]:
X_train_sequences = senetences_to_indices(X_train,words_to_index,max_len = max_len)
X_train[1],X_train_sequences[1]

('I am proud of your achievements',
 array([185457.,  52943., 293982., 268046., 394565.,  45460.,      0.,
             0.,      0.,      0.]))

In [26]:
def pretrained_embedding_layer(embeddings_index,words_to_index):
  vocab_len = len(words_to_index) + 1
  emb_dim=embeddings_index['ali'].shape[0]
  emb_matrix=np.zeros((vocab_len,emb_dim))
  for word,index in words_to_index.items():
    emb_matrix[index,:]=embeddings_index[word]
  embedding_layer = Embedding(vocab_len,emb_dim,trainable=False)
  embedding_layer.build((None,))
  embedding_layer.set_weights([emb_matrix])
  return embedding_layer

## Model1:

In [27]:
def EmojifyModel(input_shape,embeddings_index,words_to_index,max_len=max_len):
  sentence_indices = Input(input_shape,dtype = np.int32)
  embedding_layer = pretrained_embedding_layer(embeddings_index,words_to_index)
  embeddings = embedding_layer(sentence_indices)

  x = AveragePooling1D(pool_size=max_len)(embeddings)
  x = Flatten()(x)
  x = Dense(5,activation='softmax')(x)
  model = Model(sentence_indices,x)
  return model

In [28]:
model = EmojifyModel((max_len,),embeddings_index,words_to_index)
model.summary()

2025-02-16 16:35:40.641299: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-16 16:35:40.641357: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-16 16:35:40.641396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (milad-HP): /proc/driver/nvidia/version does not exist
2025-02-16 16:35:40.642000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-16 16:35:40.711408: W tensorflow/tsl/framework/cpu_allocator_impl.c

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 100)           40000100  
                                                                 
 average_pooling1d (AverageP  (None, 1, 100)           0         
 ooling1D)                                                       
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 5)                 505       
                                                                 
Total params: 40,000,605
Trainable params: 505
Non-trainable params: 40,000,100
_______________________________________________

2025-02-16 16:35:41.595917: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 160000400 exceeds 10% of free system memory.


In [29]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [31]:
model.fit(X_train_sequences,Y_train_oh,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
5/5 [==============================] - 0s 11ms/step - loss: 1.5187 - accuracy: 0.3333
Epoch 2/50
5/5 [==============================] - 0s 8ms/step - loss: 1.5156 - accuracy: 0.3333
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: 1.5130 - accuracy: 0.3258
Epoch 4/50
5/5 [==============================] - 0s 5ms/step - loss: 1.5106 - accuracy: 0.3182
Epoch 5/50
5/5 [==============================] - 0s 5ms/step - loss: 1.5081 - accuracy: 0.3258
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 1.5062 - accuracy: 0.3333
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 1.5034 - accuracy: 0.3333
Epoch 8/50
5/5 [==============================] - 0s 7ms/step - loss: 1.5005 - accuracy: 0.3712
Epoch 9/50
5/5 [==============================] - 0s 7ms/step - loss: 1.4985 - accuracy: 0.3561
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4962 - accuracy: 0.3636
Epoch 11/50
5/5 [=====================

In [32]:
x_test = np.array(['I am happy','not feeling happy','i want to die'])
x_test_indices = senetences_to_indices(x_test,words_to_index,max_len)
pred = model.predict(x_test_indices)

for item,label in zip(x_test,pred):
  print(item,label_to_emoji(np.argmax(label)))

1/1 [==============================] - 0s 127ms/step
I am happy 😄
not feeling happy 😄
i want to die 😞


## Model2:

In [34]:
def Emojify_V2(input_shape,word_to_vec_map,word_to_index):
  sentence_indices = Input(input_shape,dtype = np.int32)
  emb = pretrained_embedding_layer(word_to_vec_map,word_to_index)
  embeddings = emb(sentence_indices)

  X = LSTM(64,return_sequences=True)(embeddings)
  X = LSTM(64)(X)
  X = Dense(5,activation='softmax')(X)

  model = Model(sentence_indices,X)
  return model

In [35]:
model2 = Emojify_V2((max_len,),embeddings_index,words_to_index)

2025-02-16 16:39:44.721368: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 160000400 exceeds 10% of free system memory.


In [36]:
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 10, 100)           40000100  
                                                                 
 lstm (LSTM)                 (None, 10, 64)            42240     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 40,075,689
Trainable params: 75,589
Non-trainable params: 40,000,100
_________________________________________________________________


In [37]:
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [38]:
model2.fit(X_train_sequences,Y_train_oh,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
5/5 [==============================] - 7s 23ms/step - loss: 1.5980 - accuracy: 0.3106
Epoch 2/50
5/5 [==============================] - 0s 26ms/step - loss: 1.5188 - accuracy: 0.3561
Epoch 3/50
5/5 [==============================] - 0s 20ms/step - loss: 1.4621 - accuracy: 0.3712
Epoch 4/50
5/5 [==============================] - 0s 21ms/step - loss: 1.4080 - accuracy: 0.4015
Epoch 5/50
5/5 [==============================] - 0s 24ms/step - loss: 1.3264 - accuracy: 0.5000
Epoch 6/50
5/5 [==============================] - 0s 21ms/step - loss: 1.2494 - accuracy: 0.5152
Epoch 7/50
5/5 [==============================] - 0s 23ms/step - loss: 1.1257 - accuracy: 0.5758
Epoch 8/50
5/5 [==============================] - 0s 23ms/step - loss: 1.0004 - accuracy: 0.6515
Epoch 9/50
5/5 [==============================] - 0s 21ms/step - loss: 0.8503 - accuracy: 0.6894
Epoch 10/50
5/5 [==============================] - 0s 20ms/step - loss: 0.7460 - accuracy: 0.7652
Epoch 11/50
5/5 [============

In [39]:
x_test = np.array(['I am happy','not feeling happy','i want to die'])
x_test_indices = senetences_to_indices(x_test,words_to_index,max_len)
pred = model.predict(x_test_indices)
for item,label in zip(x_test,pred):
  print(item,label_to_emoji(np.argmax(label)))

1/1 [==============================] - 0s 44ms/step
I am happy 😄
not feeling happy 😄
i want to die 😞


## Model3:

In [41]:
def Emojify_V3(input_shape,word_to_vec_map,word_to_index):
  sentence_indices = Input(input_shape,dtype=np.int32)
  emb = pretrained_embedding_layer(word_to_vec_map,word_to_index)
  embeddings = emb(sentence_indices)

  X = Bidirectional(LSTM(64,return_sequences=True))(embeddings)
  X = Bidirectional(LSTM(64))(X)
  X = Dense(5,activation='softmax')(X)

  model = Model(sentence_indices,X)
  return model

In [42]:
model3 = Emojify_V3((max_len,),embeddings_index,words_to_index)
model3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 10, 100)           40000100  
                                                                 
 bidirectional (Bidirectiona  (None, 10, 128)          84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
Total params: 40,184,041
Trainable params: 183,941
Non-trai

In [43]:
model3.fit(X_train_sequences,Y_train_oh,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
5/5 [==============================] - 11s 29ms/step - loss: 1.6025 - accuracy: 0.2500
Epoch 2/50
5/5 [==============================] - 0s 26ms/step - loss: 1.4874 - accuracy: 0.4015
Epoch 3/50
5/5 [==============================] - 0s 37ms/step - loss: 1.4136 - accuracy: 0.4924
Epoch 4/50
5/5 [==============================] - 0s 45ms/step - loss: 1.3114 - accuracy: 0.5758
Epoch 5/50
5/5 [==============================] - 0s 55ms/step - loss: 1.2047 - accuracy: 0.5909
Epoch 6/50
5/5 [==============================] - 0s 33ms/step - loss: 1.0395 - accuracy: 0.7424
Epoch 7/50
5/5 [==============================] - 0s 32ms/step - loss: 0.8610 - accuracy: 0.7273
Epoch 8/50
5/5 [==============================] - 0s 32ms/step - loss: 0.7570 - accuracy: 0.7500
Epoch 9/50
5/5 [==============================] - 0s 41ms/step - loss: 0.5717 - accuracy: 0.8333
Epoch 10/50
5/5 [==============================] - 0s 57ms/step - loss: 0.4728 - accuracy: 0.8485
Epoch 11/50
5/5 [===========

In [44]:
x_test = np.array(['I am happy','not feeling happy','i want to die'])
x_test_indices = senetences_to_indices(x_test,words_to_index,max_len)
pred = model.predict(x_test_indices)
for item,label in zip(x_test,pred):
  print(item,label_to_emoji(np.argmax(label)))

1/1 [==============================] - 0s 42ms/step
I am happy 😄
not feeling happy 😄
i want to die 😞
